<a href="https://colab.research.google.com/github/Sourodip-Kundu/Resume_Parser/blob/main/NLP_Resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Importing required Libraries
import pandas as pd
import math
import numpy as np
import os
import json
import random
import logging
import re
import spacy
import random

In [ ]:
data_file_address = r"/content/traindata.json"

In [ ]:
# Reading data
df_data = pd.read_json(data_file_address, lines=True)
df_data.head(10)

,content,annotation
0,Govardhana K\nSenior Software Engineer\n\nBeng...,"[{'label': ['Companies worked at'], 'points': ..."
1,"Harini Komaravelli\nTest Analyst at Oracle, Hy...","[{'label': ['Companies worked at'], 'points': ..."
2,Hartej Kathuria\nData Analyst Intern - Oracle ...,"[{'label': ['Skills'], 'points': [{'start': 22..."
3,Ijas Nizamuddin\nAssociate Consultant - State ...,"[{'label': ['Skills'], 'points': [{'start': 46..."
4,"Imgeeyaul Ansari\njava developer\n\nPune, Maha...","[{'label': ['Skills'], 'points': [{'start': 18..."
5,"Jay Madhavi\nNavi Mumbai, Maharashtra - Email ...","[{'label': ['Graduation Year'], 'points': [{'s..."
6,Jitendra Babu\nFI/CO Consultant in Tech Mahind...,"[{'label': ['Graduation Year'], 'points': [{'s..."
7,Jyotirbindu Patnaik\nAssociate consultant@SAP ...,"[{'label': ['Skills'], 'points': [{'start': 30..."
8,Karthihayini C\nSystems Engineer - Infosys Lim...,"[{'label': ['Skills'], 'points': [{'start': 21..."
9,Karthik GV\nArchitect - Microsoft India\n\nHyd...,"[{'label': ['Skills'], 'points': [{'start': 40..."


In [ ]:
# Removing New Line characters
for i in range(len(df_data)):
    df_data["content"][i] = df_data["content"][i].replace("\n", " ")
df_data.head()

,content,annotation
0,Govardhana K Senior Software Engineer Bengalu...,"[{'label': ['Companies worked at'], 'points': ..."
1,"Harini Komaravelli Test Analyst at Oracle, Hyd...","[{'label': ['Companies worked at'], 'points': ..."
2,Hartej Kathuria Data Analyst Intern - Oracle R...,"[{'label': ['Skills'], 'points': [{'start': 22..."
3,Ijas Nizamuddin Associate Consultant - State S...,"[{'label': ['Skills'], 'points': [{'start': 46..."
4,"Imgeeyaul Ansari java developer Pune, Maharas...","[{'label': ['Skills'], 'points': [{'start': 18..."


In [ ]:
# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    #only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']
                        
                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1 , label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data        

In [ ]:
data = trim_entity_spans(convert_dataturks_to_spacy(data_file_address))
data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
train_data = data

In [ ]:
nlp = spacy.blank('en')

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)

In [ ]:
train_model(train_data)

Statring iteration 0
{'ner': 16059.321009889296}
Statring iteration 1
{'ner': 19154.564969066312}
Statring iteration 2
{'ner': 14560.293697806977}
Statring iteration 3
{'ner': 9291.45292547584}
Statring iteration 4
{'ner': 7562.01529594919}
Statring iteration 5
{'ner': 6327.038499304456}
Statring iteration 6
{'ner': 6229.796808332896}
Statring iteration 7
{'ner': 8138.45386882096}
Statring iteration 8
{'ner': 5211.816256031629}
Statring iteration 9
{'ner': 4901.213735437366}


In [ ]:
nlp.to_disk('nlp_model')

In [ ]:
nlp_model = spacy.load('nlp_model')

In [ ]:
train_data[0][0]

'Nida Khan Tech Support Executive - Teleperformance for Microsoft  Jaipur, Rajasthan - Email me on Indeed: indeed.com/r/Nida-Khan/6c9160696f57efd8  • To be an integral part of the organization and enhance my knowledge to utilize it in a productive manner for the growth of the company and the global.  INDUSTRIAL TRAINING  • BHEL, (HEEP) HARIDWAR On CNC System&amp; PLC Programming.  WORK EXPERIENCE  Tech Support Executive  Teleperformance for Microsoft -  September 2017 to Present  process. • 21 months of experience in ADFC as Phone Banker.  EDUCATION  Bachelor of Technology in Electronics & communication Engg  GNIT institute of Technology -  Lucknow, Uttar Pradesh  2008 to 2012  Class XII  U.P. Board -  Bareilly, Uttar Pradesh  2007  Class X  U.P. Board -  Bareilly, Uttar Pradesh  2005  SKILLS  Microsoft office, excel, cisco, c language, cbs. (4 years)  https://www.indeed.com/r/Nida-Khan/6c9160696f57efd8?isid=rex-download&ikw=download-top&co=IN'

In [ ]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Nida Khan
DESIGNATION                   - Tech Support Executive
COMPANIES WORKED AT           - Microsoft
EMAIL ADDRESS                 - indeed.com/r/Nida-Khan/6c9160696f57efd8
DESIGNATION                   - Tech Support Executive
DEGREE                        - Bachelor of Technology in Electronics & communication Engg
COLLEGE NAME                  - GNIT institute of Technology
GRADUATION YEAR               - 2012
SKILLS                        - Microsoft office, excel, cisco, c language, cbs. (4 years)
